In [9]:
import h5py
import pickle
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.svm import SVR
from statsmodels.tsa.arima.model import ARIMA
import math
import numpy.linalg as la
from tqdm import tqdm
from tabulate import tabulate

# Загрузка данных
with h5py.File('../data/raw_data/METR-LA/METR-LA.h5', 'r') as file:
    axis0 = file['df']['axis0'][:]              # Идентификаторы датчиков
    axis1 = file['df']['axis1'][:]              # Метки времени
    timestamps = pd.to_datetime(axis1)          # Преобразование меток времени в формат datetime
    df_data = file['df']['block0_values'][:]    # Данные замеров скорости

axis0 = [x.decode('utf-8') for x in axis0]
metr_la = pd.DataFrame(df_data, index=timestamps, columns=axis0)
metr_la = metr_la.iloc[:2016, :2]

# Загрузка матрицы смежности
with open('../data/raw_data/METR-LA/adj_METR-LA.pkl', 'rb') as file:
    data = pickle.load(file, encoding='bytes')

node_ids = [x.decode('utf-8') for x in data[0]]  # Получаем список id узлов из data[0]
adj_matrix = data[2]  # Получаем матрицу смежности из data[2]
metr_la_adj = pd.DataFrame(adj_matrix, index=node_ids, columns=node_ids)  # Создание DataFrame

In [34]:
# Предобработка данных (ваша функция)
def preprocess_data(data, time_len, rate, seq_len, pre_len):
    data = np.array(data)
    train_size = int(time_len * rate)
    train_data = data[:train_size]
    test_data = data[train_size:time_len]

    trainX, trainY, testX, testY = [], [], [], []
    for i in range(len(train_data) - seq_len - pre_len):
        a = train_data[i: i + seq_len + pre_len]
        trainX.append(a[0: seq_len])
        trainY.append(a[seq_len: seq_len + pre_len])
    for i in range(len(test_data) - seq_len - pre_len):
        b = test_data[i: i + seq_len + pre_len]
        testX.append(b[0: seq_len])
        testY.append(b[seq_len: seq_len + pre_len])
    return np.array(trainX), np.array(trainY), np.array(testX), np.array(testY)

# Параметры
time_len = len(metr_la)
rate = 0.8
seq_len = 12
pre_len = 3

# Предобработка
trainX, trainY, testX, testY = preprocess_data(metr_la.values, time_len, rate, seq_len, pre_len)

# Преобразование в тензоры PyTorch
# trainX = torch.tensor(trainX, dtype=torch.float32).to(device)
# trainY = torch.tensor(trainY, dtype=torch.float32).to(device)
# testX = torch.tensor(testX, dtype=torch.float32).to(device)
# testY = torch.tensor(testY, dtype=torch.float32).to(device)

from sklearn.preprocessing import MinMaxScaler

# Нормализация данных
def normalize_data(trainX, testX):
    scaler = MinMaxScaler()
    # Обучение scaler на тренировочных данных и преобразование всех данных
    trainX_shape = trainX.shape
    testX_shape = testX.shape
    trainX = scaler.fit_transform(trainX.reshape(-1, trainX_shape[-1])).reshape(trainX_shape)
    testX = scaler.transform(testX.reshape(-1, testX_shape[-1])).reshape(testX_shape)
    return trainX, testX, scaler

# Нормализация trainX и testX
trainX_norm, testX_norm, scaler_X = normalize_data(trainX, testX)
trainY_norm, testY_norm, scaler_Y = normalize_data(trainY, testY)

# Преобразование в тензоры PyTorch
trainX_norm = torch.tensor(trainX_norm, dtype=torch.float32).to(device)
trainY_norm = torch.tensor(trainY_norm, dtype=torch.float32).to(device)
testX_norm = torch.tensor(testX_norm, dtype=torch.float32).to(device)
testY_norm = torch.tensor(testY_norm, dtype=torch.float32).to(device)


In [23]:
# LSTM
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1, pre_len=3):
        super(LSTMModel, self).__init__()
        self.pre_len = pre_len
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h_lstm, _ = self.lstm(x)  # h_lstm shape: [batch, seq_len, hidden_size]
        # Используем последние pre_len скрытые состояния
        h_lstm = h_lstm[:, -self.pre_len:, :]  # shape: [batch, pre_len, hidden_size]
        out = self.fc(h_lstm)  # shape: [batch, pre_len, output_size]
        return out

In [24]:
class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1, pre_len=3):
        super(GRUModel, self).__init__()
        self.pre_len = pre_len
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h_gru, _ = self.gru(x)  # h_gru shape: [batch, seq_len, hidden_size]
        # Используем последние pre_len скрытые состояния
        h_gru = h_gru[:, -self.pre_len:, :]  # shape: [batch, pre_len, hidden_size]
        out = self.fc(h_gru)  # shape: [batch, pre_len, output_size]
        return out


In [25]:
class TCNModel(nn.Module):
    def __init__(self, input_size, output_size, num_channels, kernel_size, dropout=0.2, pre_len=3):
        super(TCNModel, self).__init__()
        self.pre_len = pre_len
        self.tcn = nn.Sequential(
            nn.Conv1d(input_size, num_channels, kernel_size, padding=(kernel_size - 1)),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Conv1d(num_channels, num_channels, kernel_size, padding=(kernel_size - 1)),
            nn.ReLU(),
            nn.Dropout(dropout),
        )
        self.fc = nn.Linear(num_channels, output_size)

    def forward(self, x):
        x = x.permute(0, 2, 1)  # (batch, seq_len, input_size) -> (batch, input_size, seq_len)
        out = self.tcn(x)  # shape: [batch, num_channels, seq_len + kernel_size - 1]
        # Используем последние pre_len временных шагов
        out = out[:, :, -self.pre_len:]  # shape: [batch, num_channels, pre_len]
        out = out.permute(0, 2, 1)  # shape: [batch, pre_len, num_channels]
        out = self.fc(out)  # shape: [batch, pre_len, output_size]
        return out


In [45]:
def train_model(model, trainX, trainY, testX, testY, scaler_Y, epochs=50, lr=0.001):
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    model.to(device)

    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(trainX)  # outputs shape: [batch, pre_len, output_size]
        loss = criterion(outputs, trainY)
        loss.backward()
        optimizer.step()

        if (epoch + 1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

    # Оценка модели
    model.eval()
    with torch.no_grad():
        predictions_norm = model(testX).cpu().numpy()  # predictions shape: [batch, pre_len, output_size]

    # Восстановление данных
    predictions = scaler_Y.inverse_transform(predictions_norm.reshape(-1, predictions_norm.shape[-1]))
    predictions = predictions.reshape(predictions_norm.shape)  # Восстановление исходной формы
    actuals = scaler_Y.inverse_transform(testY.cpu().numpy().reshape(-1, testY.shape[-1]))
    actuals = actuals.reshape(testY.shape)  # Восстановление исходной формы

    # Метрики
    rmse = np.sqrt(mean_squared_error(actuals.reshape(-1), predictions.reshape(-1)))
    mae = mean_absolute_error(actuals.reshape(-1), predictions.reshape(-1))
    r2 = r2_score(actuals.reshape(-1), predictions.reshape(-1))
    var = np.var(actuals - predictions)
    return rmse, mae, r2, var

In [36]:
# Параметры моделей
input_size = trainX_norm.shape[2]  # Количество узлов (features)
output_size = trainY_norm.shape[2]  # Количество узлов (features)
hidden_size = 64
num_layers = 2
num_channels = 64
kernel_size = 3
pre_len = trainY_norm.shape[1]  # Длина предсказания (pre_len)

# Создание моделей
lstm_model = LSTMModel(input_size, hidden_size, output_size, num_layers, pre_len)
gru_model = GRUModel(input_size, hidden_size, output_size, num_layers, pre_len)
tcn_model = TCNModel(input_size, output_size, num_channels, kernel_size, pre_len=pre_len)


In [40]:
# Обучение и оценка
results = []
for name, model in [("LSTM", lstm_model), ("GRU", gru_model), ("TCN", tcn_model)]:
    rmse, mae, r2, var = train_model(model, trainX_norm, trainY_norm, testX_norm, testY_norm, scaler_Y, epochs=1000)
    results.append([name, rmse, mae, r2, var])

# Вывод результатов
print(tabulate(results, headers=["Method", "RMSE", "MAE", "R2", "Var"]))

Epoch [50/1000], Loss: 0.0164
Epoch [100/1000], Loss: 0.0158
Epoch [150/1000], Loss: 0.0154
Epoch [200/1000], Loss: 0.0149
Epoch [250/1000], Loss: 0.0144
Epoch [300/1000], Loss: 0.0141
Epoch [350/1000], Loss: 0.0142
Epoch [400/1000], Loss: 0.0137
Epoch [450/1000], Loss: 0.0136
Epoch [500/1000], Loss: 0.0135
Epoch [550/1000], Loss: 0.0134
Epoch [600/1000], Loss: 0.0140
Epoch [650/1000], Loss: 0.0133
Epoch [700/1000], Loss: 0.0132
Epoch [750/1000], Loss: 0.0131
Epoch [800/1000], Loss: 0.0130
Epoch [850/1000], Loss: 0.0129
Epoch [900/1000], Loss: 0.0127
Epoch [950/1000], Loss: 0.0126
Epoch [1000/1000], Loss: 0.0130
Epoch [50/1000], Loss: 0.0140
Epoch [100/1000], Loss: 0.0139
Epoch [150/1000], Loss: 0.0138
Epoch [200/1000], Loss: 0.0136
Epoch [250/1000], Loss: 0.0135
Epoch [300/1000], Loss: 0.0134
Epoch [350/1000], Loss: 0.0133
Epoch [400/1000], Loss: 0.0131
Epoch [450/1000], Loss: 0.0130
Epoch [500/1000], Loss: 0.0128
Epoch [550/1000], Loss: 0.0127
Epoch [600/1000], Loss: 0.0125
Epoch [65

In [46]:
from sklearn.model_selection import ParameterGrid

# Определение сетки параметров для каждой модели
param_grid = {
    "LSTM": {
        "hidden_size": [32, 64, 128],
        "num_layers": [4],
        "lr": [0.002],
        "epochs": [100, 300]
    },
    "GRU": {
        "hidden_size": [64, 128],
        "num_layers": [4],
        "lr": [0.002],
        "epochs": [100, 300]
    },
    "TCN": {
        "num_channels": [32, 64, 128],
        "kernel_size": [4],
        "lr": [0.002],
        "epochs": [100, 300]
    }
}

# Функция для Grid Search
def grid_search(model_class, model_name, param_grid, trainX, trainY, testX, testY, scaler_Y):
    best_rmse = float("inf")
    best_params = None
    results = []

    # Перебор всех комбинаций параметров
    for params in ParameterGrid(param_grid):
        # print(f"Testing {model_name} with params: {params}")

        # Создание модели
        if model_name == "LSTM":
            model = model_class(
                input_size=trainX.shape[2],
                hidden_size=params["hidden_size"],
                output_size=trainY.shape[2],
                num_layers=params["num_layers"],
                pre_len=trainY.shape[1]
            )
        elif model_name == "GRU":
            model = model_class(
                input_size=trainX.shape[2],
                hidden_size=params["hidden_size"],
                output_size=trainY.shape[2],
                num_layers=params["num_layers"],
                pre_len=trainY.shape[1]
            )
        elif model_name == "TCN":
            model = model_class(
                input_size=trainX.shape[2],
                output_size=trainY.shape[2],
                num_channels=params["num_channels"],
                kernel_size=params["kernel_size"],
                pre_len=trainY.shape[1]
            )

        # Обучение модели
        rmse, mae, r2, var = train_model(
            model, trainX, trainY, testX, testY, scaler_Y,
            epochs=params["epochs"], lr=params["lr"]
        )

        # Сохранение результатов
        results.append({
            "params": params,
            "rmse": rmse,
            "mae": mae,
            "r2": r2,
            "var": var
        })

        # Обновление лучших параметров
        if rmse < best_rmse:
            best_rmse = rmse
            best_params = params

    return best_params, best_rmse, results

# Выполнение Grid Search для каждой модели
best_results = {}
for model_name, model_class in [("LSTM", LSTMModel), ("GRU", GRUModel), ("TCN", TCNModel)]:
    best_params, best_rmse, results = grid_search(
        model_class, model_name, param_grid[model_name],
        trainX_norm, trainY_norm, testX_norm, testY_norm, scaler_Y
    )
    best_results[model_name] = {
        "best_params": best_params,
        "best_rmse": best_rmse,
        "all_results": results
    }

# Вывод лучших параметров и результатов
for model_name, result in best_results.items():
    print(f"Best params for {model_name}: {result['best_params']}")
    print(f"Best RMSE for {model_name}: {result['best_rmse']}")
    print("All results:")
    print(tabulate(result["all_results"], headers="keys"))

Epoch [100/100], Loss: 0.0254
Epoch [100/100], Loss: 0.0257
Epoch [100/100], Loss: 0.0235
Epoch [100/300], Loss: 0.0254
Epoch [200/300], Loss: 0.0191
Epoch [300/300], Loss: 0.0150
Epoch [100/300], Loss: 0.0257
Epoch [200/300], Loss: 0.0231
Epoch [300/300], Loss: 0.0179
Epoch [100/300], Loss: 0.0254
Epoch [200/300], Loss: 0.0183
Epoch [300/300], Loss: 0.0136
Epoch [100/100], Loss: 0.0203
Epoch [100/100], Loss: 0.0169
Epoch [100/300], Loss: 0.0198
Epoch [200/300], Loss: 0.0152
Epoch [300/300], Loss: 0.0142
Epoch [100/300], Loss: 0.0181
Epoch [200/300], Loss: 0.0144
Epoch [300/300], Loss: 0.0136
Epoch [100/100], Loss: 0.0393
Epoch [100/100], Loss: 0.0297
Epoch [100/100], Loss: 0.0191
Epoch [100/300], Loss: 0.0416
Epoch [200/300], Loss: 0.0319
Epoch [300/300], Loss: 0.0263
Epoch [100/300], Loss: 0.0239
Epoch [200/300], Loss: 0.0202
Epoch [300/300], Loss: 0.0184
Epoch [100/300], Loss: 0.0207
Epoch [200/300], Loss: 0.0177
Epoch [300/300], Loss: 0.0167
Best params for LSTM: {'epochs': 100, 'h